In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf

X = np.load('x_letters.npy')
y = np.load('y_letters.npy')

X_tensor = tf.convert_to_tensor(X, dtype=tf.float32)

y_tensor = tf.convert_to_tensor(y, dtype=tf.float32)

y_flat = tf.reshape(y_tensor, [-1])

vals, idx, counts = tf.unique_with_counts(y_flat)

first_indices = []
for v in vals.numpy():
    pos = tf.where(y_flat == v)
    first_indices.append(pos[0][0].numpy())

for i, val, count in zip(first_indices, vals.numpy(), counts.numpy()):
    print(f"Value: {val}, Count: {count}")
    img = X[i]
    #plt.imshow(img, cmap='gray')
    # plt.axis('off')
    # plt.show()

In [ ]:
import keras
from tensorflow.keras import layers, Sequential
import string


@keras.saving.register_keras_serializable()
class Descriminator(tf.keras.Model):
    def __init__(self):
        super(Descriminator, self).__init__()
        self.dense = Sequential([
            layers.Dense(256, activation="relu"),
            layers.Dropout(0.3),
            layers.Dense(128, activation="relu"),
            layers.Dropout(0.3),
            layers.Dense(64, activation="relu"),
            layers.Dropout(0.3),
        ])
        self.out_real_classifier = layers.Dense(1)
        self.out_label_classifier = layers.Dense(26)

    def call(self, x):
        x = tf.reshape(x, [x.shape[0], -1])
        x = self.dense(x)
        reality_logits = self.out_real_classifier(x)
        label_logits = self.out_label_classifier(x)
        return reality_logits, label_logits

    def get_config(self):
        return {}

    @classmethod
    def from_config(cls, config):
        return cls()


@keras.saving.register_keras_serializable()
class Generator(tf.keras.Model):
    def __init__(self):
        super(Generator, self).__init__()
        self.net = Sequential([
            layers.Dense(14 * 14, activation="relu"),
            layers.Dropout(0.3),
            layers.Dense(28 * 28, activation="sigmoid"),
            layers.Dropout(0.3),
        ])

    def call(self, noise, labels):
        x = tf.concat([noise, labels], axis=1)
        x = self.net(x)
        x = tf.reshape(x, [-1, 28, 28])
        return x


def discriminatorLoss(real_reality_logits, fake_reality_logits, labels_logits, labels_gt):
    bce = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    sce = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

    real_loss = bce(tf.ones_like(real_reality_logits),
                    real_reality_logits)
    fake_loss = bce(tf.zeros_like(fake_reality_logits),
                    fake_reality_logits)
    label_loss = sce(labels_gt, labels_logits)

    total_loss = real_loss + fake_loss
    return total_loss


def generatorLoss(fake_logits, labels_logits, labels_gt):
    bce = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    sce = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

    real_loss = bce(tf.ones_like(fake_logits), fake_logits)
    label_loss = sce(labels_gt, labels_logits) * 1.1
    total_loss = real_loss
    return total_loss


import tensorflow as tf


def generate_random_image(generator, latent_dim):
    random_latent_vector = tf.random.normal(shape=(1, latent_dim))
    random_label_index = tf.random.uniform(shape=(1,), minval=0, maxval=26, dtype=tf.int32)

    random_label = tf.one_hot(random_label_index, depth=26)

    generated_image = generator(random_latent_vector, random_label)

    letter = string.ascii_uppercase[random_label_index[0].numpy()]

    plt.imshow(generated_image[0], cmap='gray')
    plt.title(f"Generated Letter: {letter}")
    plt.axis('off')
    plt.show()


def generate_all_letters(generator, latent_dim, img_size=(28, 28)):
    num_letters = 26
    plt.figure(figsize=(22, 20))

    for i, letter in enumerate(string.ascii_uppercase):
        latent_vector = tf.random.normal(shape=(1, latent_dim))
        label = tf.one_hot([i], depth=26)
        generated_image = generator(latent_vector, label)

        plt.subplot(1, num_letters, i + 1)
        plt.imshow(generated_image[0], cmap='gray')
        plt.axis('off')
        plt.title(letter, fontsize=8)

    plt.tight_layout()
    plt.show()

In [ ]:
from tensorflow.keras import layers, optimizers
import keras
from keras import ops
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'


def train_d(batch_size, real_images, real_labels, discriminator, generator, d_optimizer, latent_dim):
    random_latent_vectors = keras.random.normal(
        shape=(batch_size, latent_dim)
    )

    random_label_indices = tf.random.uniform(
        shape=(batch_size,),
        minval=0,
        maxval=26,
        dtype=tf.int32
    )

    random_labels = tf.one_hot(random_label_indices, depth=26)

    generated_images = generator(random_latent_vectors, random_labels)

    with tf.GradientTape() as tape:
        all_images = tf.concat([real_images, generated_images], axis=0)

        all_reality_logits, all_label_logits = discriminator(all_images, training=True)

        reality_real_logits, reality_generated_logits = tf.split(all_reality_logits,
                                                                 num_or_size_splits=[batch_size, batch_size],
                                                                 axis=0)
        reality_label_logits, label_generated_logits = tf.split(all_label_logits,
                                                                num_or_size_splits=[batch_size, batch_size],
                                                                axis=0)
        loss = discriminatorLoss(reality_real_logits, reality_generated_logits, reality_label_logits,
                                 real_labels)
        grads = tape.gradient(loss, discriminator.trainable_variables)
        d_optimizer.apply_gradients(zip(grads, discriminator.trainable_variables))
    return loss


def train_g(batch_size, discriminator, generator, g_optimizer, latent_dim, real_images):
    random_latent_vectors = keras.random.normal(
        shape=(batch_size, latent_dim)
    )

    random_label_indices = tf.random.uniform(
        shape=(batch_size,),
        minval=0,
        maxval=26,
        dtype=tf.int32
    )

    random_labels = tf.one_hot(random_label_indices, depth=26)

    with tf.GradientTape() as tape:
        reality_generated_logits, label_genarated_logits = discriminator(
            generator(random_latent_vectors, random_labels, training=True), training=False)
        g_loss = generatorLoss(reality_generated_logits, label_genarated_logits, random_label_indices)
    grads = tape.gradient(g_loss, generator.trainable_weights)
    g_optimizer.apply_gradients(zip(grads, generator.trainable_weights))

    return g_loss


def should_restart(last_loss, current_loss, threshold=0.01, max_epochs_no_improve=5, current_epoch_no_improve=0):
    improvement = last_loss - current_loss
    if improvement > threshold:
        current_epoch_no_improve = 0
    else:
        current_epoch_no_improve += 1

    if current_epoch_no_improve >= max_epochs_no_improve:
        return True, current_epoch_no_improve
    else:
        return False, current_epoch_no_improve


def save_discriminator(discriminator):
    discriminator.save('best_discriminator_model.keras')
    print("Saving discriminator and optimizer to memory...")


def save_generator(generator):
    generator.save('best_generator_model.keras')
    print("Saving generator and optimizer to memory...")


def load_discriminator():
    discriminator = tf.keras.models.load_model('best_discriminator_model.keras')
    return discriminator


def load_generator():
    generator = tf.keras.models.load_model('best_generator_model.keras')
    return generator


discriminator = Descriminator()
d_optimizer = optimizers.Adam(learning_rate=1e-4)
generator = Generator()
g_optimizer = optimizers.Adam(learning_rate=1e-4)

latent_dim = 100
d_LOSS_DELTA = 0.01
g_LOSS_DELTA = 0.01
d_min_loss = 2.0
g_min_loss = 2.0
max_iter_d = 500
max_iter_g = 1000
cycle = 500000

dataset = tf.data.Dataset.from_tensor_slices((X_tensor, y_tensor))
print(len(dataset))
dataset = dataset.shuffle(buffer_size=1024)
dataset = dataset.batch(30000)
dataset = dataset.prefetch(tf.data.AUTOTUNE)
reload_g = False
reload_d = False
skip_d = False


In [ ]:
for c in range(cycle):
    g_loss = 0
    d_loss = 0
    d_epoch = 0
    g_epoch = 0
    while True:
        d_epoch += 1
        for real_images, real_labels in dataset:
            batch_size = ops.shape(real_images)[0]
            d_loss = train_d(batch_size, real_images, real_labels,
                                 discriminator, generator, d_optimizer, latent_dim).numpy()
            if d_loss < d_min_loss:
                break
        if d_loss < d_min_loss:
            break

        if d_epoch % 100 == 0:
            print("Still training discriminator, epoch:", d_epoch, ", d_loss:", d_loss)


    while True:
        g_epoch += 1
        for real_images, real_labels in dataset:
            batch_size = ops.shape(real_images)[0]
            g_loss = train_g(batch_size, discriminator, generator, g_optimizer, latent_dim).numpy()

            if g_loss < g_min_loss:
                break
        if g_loss < g_min_loss:
            break

        if g_epoch % 100 == 0:
            print("Still training generator, epoch:", g_epoch, ", g_loss:", g_loss)
    if c % 1 == 0:
        print("epoch:", g_epoch, ", g_loss:", g_loss, "d_loss:", d_loss)
        save_discriminator(discriminator)
        save_generator(generator)
        generate_all_letters(generator, latent_dim)

    d_min_loss = d_min_loss - d_min_loss * 0.05
    g_min_loss = g_min_loss - g_min_loss * 0.05


In [ ]:
import random

first_run = True
for c in range(cycle):
    g_loss = 0
    d_loss = 0
    d_epoch = 0
    g_epoch = 0

    for real_images, real_labels in dataset:
        batch_size = ops.shape(real_images)[0]
        num = random.randint(1, 100)
        if num < 70:
            d_loss = train_d(batch_size, real_images, real_labels,
                             discriminator, generator, d_optimizer, latent_dim).numpy()
        g_loss = train_g(batch_size, discriminator, generator, g_optimizer, latent_dim).numpy()

    if c % 1 == 0:
        print("epoch:", g_epoch, ", g_loss:", g_loss, "d_loss:", d_loss)
        save_discriminator(discriminator)
        save_generator(generator)
        generate_all_letters(generator, latent_dim)

    d_min_loss = d_min_loss - d_min_loss * 0.05
    g_min_loss = g_min_loss - g_min_loss * 0.05


In [ ]:

import matplotlib.pyplot as plt
import tensorflow as tf
import string

letters = list(string.ascii_uppercase)

for i, val, count in zip(first_indices, vals.numpy(), counts.numpy()):
    print(f"Value: {val}, Count: {count}")
    img = X[i]

    plt.imshow(img, cmap='gray')
    plt.axis('off')
    plt.show()

    val_batch = tf.expand_dims(img, axis=0)

    reality_logits, label_logits = discriminator(val_batch)

    print("Reality logits:", reality_logits.numpy())

    probs = tf.nn.softmax(label_logits)  # convert logits to probabilities
    top3 = tf.nn.top_k(probs, k=3)

    top_letters = [letters[idx] for idx in top3.indices.numpy()[0]]
    top_probs = top3.values.numpy()[0]

    print("Top 3 predicted letters:")
    for letter, prob in zip(top_letters, top_probs):
        print(f"{letter}: {prob:.4f}")

    if i > 5:
        break

In [ ]:
import keras
from keras import layers
from keras import ops


class GAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super().__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim
        self.seed_generator = keras.random.SeedGenerator(1337)

    def compile(self, d_optimizer, g_optimizer, loss_fn_discriminator, loss_fn_generator):
        super().compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn_generator = loss_fn_generator
        self.loss_fn_discriminator = loss_fn_discriminator
        self.d_loss_metric = keras.metrics.Mean(name="d_loss")
        self.g_loss_metric = keras.metrics.Mean(name="g_loss")

    @property
    def metrics(self):
        return [self.d_loss_metric, self.g_loss_metric]

    def train_step(self, real_images, real_labels, discriminator, generator):
        batch_size = ops.shape(real_images)[0]

        if discriminator:
            random_latent_vectors = keras.random.normal(
                shape=(batch_size, self.latent_dim)
            )

            random_label_indices = tf.random.uniform(
                shape=(batch_size,),
                minval=0,
                maxval=26,
                dtype=tf.int32
            )

            random_labels = tf.one_hot(random_label_indices, depth=26)

            generated_images = self.generator(random_latent_vectors, random_labels)

            with tf.GradientTape() as tape:
                all_images = tf.concat([real_images, generated_images], axis=0)

                all_reality_logits, all_label_logits = self.discriminator(all_images, training=True)

                reality_generated_logits, reality_real_logits = tf.split(all_reality_logits,
                                                                         num_or_size_splits=[batch_size, batch_size],
                                                                         axis=0)
                label_generated_logits, reality_label_logits = tf.split(all_label_logits,
                                                                        num_or_size_splits=[batch_size, batch_size],
                                                                        axis=0)
                d_loss = self.loss_fn_discriminator(reality_real_logits, reality_generated_logits, reality_label_logits,
                                                    real_labels)
                grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
                self.d_optimizer.apply_gradients(zip(grads, self.discriminator.trainable_weights))
                for g, w in zip(grads, self.discriminator.trainable_weights):
                    print(w.name, g is None)

        if generator:
            random_latent_vectors = keras.random.normal(
                shape=(batch_size, self.latent_dim)
            )

            random_label_indices = tf.random.uniform(
                shape=(batch_size,),
                minval=0,
                maxval=26,
                dtype=tf.int32
            )

            random_labels = tf.one_hot(random_label_indices, depth=26)

            with tf.GradientTape() as tape:
                reality_generated_logits, label_genarated_logits = self.discriminator(
                    self.generator(random_latent_vectors, random_labels))
                g_loss = self.loss_fn_generator(reality_generated_logits, label_genarated_logits, random_label_indices)
            grads = tape.gradient(g_loss, self.generator.trainable_weights)
            self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        self.d_loss_metric.update_state(d_loss if discriminator else 0)
        # self.g_loss_metric.update_state(g_loss if generator else 0)
        return {
            "d_loss": self.d_loss_metric.result(),
            # "g_loss": self.g_loss_metric.result(),
        }
